In [1]:
#|default_exp toolloop

# Tool loop

In [2]:
#| export
from cosette.core import *
from fastcore.utils import *
from fastcore.meta import delegates

from openai.resources.chat import Completions

In [3]:
model = models[0]

In [4]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

In [5]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

In [6]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model, tools=tools)

In [7]:
r = chat('Can you tell me the email address for customer C2?')

- Retrieving customer C2


In [8]:
choice = r.choices[0]
print(choice.finish_reason)
choice

tool_calls


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BYev4ExQk899v9yjLERyDGSc', function=Function(arguments='{"customer_id":"C2"}', name='get_customer_info'), type='function')]))

In [9]:
r = chat()
r

The email address for customer C2 (Jane Smith) is jane@example.com.

<details>

- id: chatcmpl-9R81d5i8pBSIRg5B7erJcF8t5BzKw
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The email address for customer C2 (Jane Smith) is jane@example.com.', role='assistant', function_call=None, tool_calls=None))]
- created: 1716252733
- model: gpt-4o-2024-05-13
- object: chat.completion
- system_fingerprint: fp_729ea513f7
- usage: CompletionUsage(completion_tokens=17, prompt_tokens=251, total_tokens=268)

</details>

In [10]:
chat = Chat(model, tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.choices[0].finish_reason)
find_block(r)

- Retrieving customer C1
tool_calls


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lENp0aVeTJ7W0q8SRgC7h5s9', function=Function(arguments='{"customer_id":"C1"}', name='get_customer_info'), type='function')])

In [22]:
#| exports
@patch
@delegates(Completions.create)
def toolloop(self:Chat,
             pr, # Prompt to pass to model
             max_steps=10, # Maximum number of tool requests to loop through
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kwargs):
    "Add prompt `pr` to dialog and get a response from the model, automatically following up with `tool_use` messages"
    r = self(pr, **kwargs)
    for i in range(max_steps):
        ch = r.choices[0]
        if ch.finish_reason!='tool_calls': break
        if trace_func: trace_func(r)
        r = self(**kwargs)
        if not (cont_func or noop)(self.h[-2]): break
    if trace_func: trace_func(r)
    return r

In [12]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

- Retrieving customer C1
ChatCompletion(id='chatcmpl-9R81fvatrbbrAuUjZRTPKWgntsRCx', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Yss1rLc1tU2wDkWPMGSGgdor', function=Function(arguments='{"customer_id":"C1"}', name='get_customer_info'), type='function')]))], created=1716252735, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_729ea513f7', usage=In: 157; Out: 17; Total: 174)
- Cancelling order O1
- Cancelling order O2
ChatCompletion(id='chatcmpl-9R81gjtdy8L3cUI4o46MDeeOdaaRb', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4AIzkbEYXTGNqd6uMsPLuVEB', function=Function(arguments='{"order_id": "O1"}', name='cancel_order'), type='function'), ChatCompletionMessa

Both orders for customer C1 have been successfully canceled.

<details>

- id: chatcmpl-9R81hQpOI0m1ExNidpdMrIiMx78pd
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Both orders for customer C1 have been successfully canceled.', role='assistant', function_call=None, tool_calls=None))]
- created: 1716252737
- model: gpt-4o-2024-05-13
- object: chat.completion
- system_fingerprint: fp_729ea513f7
- usage: CompletionUsage(completion_tokens=12, prompt_tokens=347, total_tokens=359)

</details>

In [13]:
chat.toolloop('What is the status of order O2?')

- Retrieving order O2


The status of order O2 is "Cancelled".

<details>

- id: chatcmpl-9R81inXsiw5xzoux1xw5Z7bBoPsuN
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The status of order O2 is "Cancelled".', role='assistant', function_call=None, tool_calls=None))]
- created: 1716252738
- model: gpt-4o-2024-05-13
- object: chat.completion
- system_fingerprint: fp_729ea513f7
- usage: CompletionUsage(completion_tokens=11, prompt_tokens=436, total_tokens=447)

</details>

## Export -

In [46]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()